In [1]:
from sleeper_wrapper import League
from sleeper_wrapper import Stats

league = League('732312711840550912')

In [2]:
d_users = league.get_users()
users = {}
for r in d_users:
    users[r['user_id']] = {'user_id': r['user_id'], 'name': r['display_name']}

In [3]:
users

{'95259455910199296': {'user_id': '95259455910199296', 'name': 'Deign'},
 '413881572480000000': {'user_id': '413881572480000000', 'name': 'anushm'},
 '467082184046407680': {'user_id': '467082184046407680', 'name': 'tejas'},
 '467082433204842496': {'user_id': '467082433204842496', 'name': 'snethi'},
 '467082675589476352': {'user_id': '467082675589476352', 'name': 'rbada'},
 '467416275337867264': {'user_id': '467416275337867264', 'name': 'vish3418'},
 '467974784802615296': {'user_id': '467974784802615296',
  'name': 'psychomaragiri'},
 '470020787244363776': {'user_id': '470020787244363776', 'name': 'preraku'},
 '471181259578535936': {'user_id': '471181259578535936', 'name': 'shwanrat'},
 '730218826838429696': {'user_id': '730218826838429696', 'name': 'JCH4'},
 '730543392060231680': {'user_id': '730543392060231680', 'name': 'suups'},
 '733202403196780544': {'user_id': '733202403196780544', 'name': 'Shang_Chi'}}

In [26]:
p = league.get_rosters()
week = 0
roster = []
new_user = {}
for i, x in enumerate(p):
    oID = x['owner_id']
    rec = users[oID]
    rec['roster_id'] = i + 1
    rec['wins'] = x['settings']['wins']
    rec['losses'] = x['settings']['losses']
    points = int(x['settings']['fpts']) + (int(x['settings']['fpts_decimal']) / 100)
    pa = int(x['settings']['fpts_against']) + (int(x['settings']['fpts_against_decimal']) / 100)
    rec['pf'] = points
    rec['pa'] = pa
    roster.append(rec['name'])
    week = rec['wins'] + rec['losses']
    if not week == 0:
        rec['avg_pf'] = points / float(week)
        rec['avg_pa'] = pa / float(week)
    else:
        rec['avg_pf'] = 0
        rec['avg_pa'] = 0
        
    new_user[rec['roster_id']] = rec

In [16]:
roster

['anushm',
 'psychomaragiri',
 'tejas',
 'rbada',
 'Deign',
 'snethi',
 'shwanrat',
 'preraku',
 'suups',
 'JCH4',
 'Shang_Chi',
 'vish3418']

In [27]:
new_user

{1: {'user_id': '413881572480000000',
  'name': 'anushm',
  'roster_id': 1,
  'wins': 4,
  'losses': 7,
  'pf': 968.34,
  'pa': 1025.9,
  'avg_pf': 88.03090909090909,
  'avg_pa': 93.26363636363637},
 2: {'user_id': '467974784802615296',
  'name': 'psychomaragiri',
  'roster_id': 2,
  'wins': 5,
  'losses': 6,
  'pf': 1130.32,
  'pa': 1120.99,
  'avg_pf': 102.75636363636363,
  'avg_pa': 101.90818181818182},
 3: {'user_id': '467082184046407680',
  'name': 'tejas',
  'roster_id': 3,
  'wins': 4,
  'losses': 7,
  'pf': 1075.7,
  'pa': 1133.56,
  'avg_pf': 97.7909090909091,
  'avg_pa': 103.05090909090909},
 4: {'user_id': '467082675589476352',
  'name': 'rbada',
  'roster_id': 4,
  'wins': 7,
  'losses': 4,
  'pf': 1210.1,
  'pa': 992.86,
  'avg_pf': 110.0090909090909,
  'avg_pa': 90.26},
 5: {'user_id': '95259455910199296',
  'name': 'Deign',
  'roster_id': 5,
  'wins': 9,
  'losses': 2,
  'pf': 1131.74,
  'pa': 1049.72,
  'avg_pf': 102.88545454545455,
  'avg_pa': 95.42909090909092},
 6: {

In [43]:
matchup_dict = {}

for i in range(1,17):
    m = league.get_matchups(i)

    matchups = {}
    for k in m:
        matchup_id = k['matchup_id']
        if matchup_id in matchups.keys():
            matchups[matchup_id].append((k['roster_id'], k['points']))
        else:
            matchups[matchup_id] = [(k['roster_id'], k['points'])]
            
    new = {}
    for j in matchups.values():
        roster_id1 = j[0][0]
        roster_id2 = j[1][0]
        one_score = j[0][1]
        two_score = j[1][1]
        one_diff = one_score - two_score
        two_diff = two_score - one_score
        
        new[roster_id1] = {
            'opp_id': roster_id2,
            'score': one_score,
            'opp_score': two_score,
            'diff': one_diff,
            'win': 1 if one_score > two_score else 0
        }
        
        new[roster_id2] = {
            'opp_id': roster_id1,
            'score': two_score,
            'opp_score': one_score,
            'diff': two_diff,
            'win': 1 if one_score > two_score else 0
        }

        
    matchup_dict[i] = new

In [44]:
print(matchup_dict)

{1: {1: {'opp_id': 6, 'score': 106.66, 'opp_score': 92.76, 'diff': 13.899999999999991, 'win': 1}, 6: {'opp_id': 1, 'score': 92.76, 'opp_score': 106.66, 'diff': -13.899999999999991, 'win': 1}, 2: {'opp_id': 3, 'score': 137.36, 'opp_score': 104.48, 'diff': 32.88000000000001, 'win': 1}, 3: {'opp_id': 2, 'score': 104.48, 'opp_score': 137.36, 'diff': -32.88000000000001, 'win': 1}, 4: {'opp_id': 12, 'score': 133.98, 'opp_score': 96.12, 'diff': 37.859999999999985, 'win': 1}, 12: {'opp_id': 4, 'score': 96.12, 'opp_score': 133.98, 'diff': -37.859999999999985, 'win': 1}, 5: {'opp_id': 8, 'score': 103.56, 'opp_score': 103.78, 'diff': -0.21999999999999886, 'win': 0}, 8: {'opp_id': 5, 'score': 103.78, 'opp_score': 103.56, 'diff': 0.21999999999999886, 'win': 0}, 7: {'opp_id': 11, 'score': 107.8, 'opp_score': 78.0, 'diff': 29.799999999999997, 'win': 1}, 11: {'opp_id': 7, 'score': 78.0, 'opp_score': 107.8, 'diff': -29.799999999999997, 'win': 1}, 9: {'opp_id': 10, 'score': 132.78, 'opp_score': 107.16, 

In [ ]:
m = league.get_matchups(16)
print(m)

In [ ]:
matchups

In [ ]:
p